# Bokeh Selector Example

This example demostrates how to use Bokeh to interactively select data from a chart within a Jupyter Notebook.

Load the required libraries. [Bokeh](http://bokeh.pydata.org/en/latest/docs/user_guide.html) is used to provide interactive plotting. The `output_notebook()` function is used to display the plots inline within a Jupyter notebook.

In [41]:
import numpy as np
from bokeh.plotting import gridplot, figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
# Direct output to this notebook
output_notebook()

Loading BokehJS ...

Generate some random data

In [25]:
n = 100
x = np.random.random(size=n) * 100
y = np.random.random(size=n) * 100
source = ColumnDataSource(data=dict(x=x, y=y))

Create another `ColumnDataSource` to store the selected data.

In [26]:
selected = ColumnDataSource(data=dict(x=[], y=[]))

Create the Bokeh plot and add tools for selecting data. The `lasso_select` tool is enabled by default. To make a multiple selection, press the SHIFT key. To clear the selection, press the ESC key. More information on the Bokeh tools is available [here](http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html). Two plots are created to simulataneously plot the dataset in two different perspectives.

In [42]:
figkwds = dict(plot_width=400, plot_height=300, webgl=True,
           tools="pan,lasso_select,box_select,help", 
           active_drag="lasso_select")

p1 = figure(**figkwds)
p1.scatter('x', 'y', source=source, alpha=0.8)

p2 = figure(**figkwds, 
            x_axis_type='log', x_range=[1, 100],
            y_axis_type='log', y_range=[1, 100])
p2.scatter('x', 'y', source=source, alpha=0.8)

p3 = figure(**figkwds)
p3.scatter('x', 'y', source=selected)
layout = gridplot([[p1, p2, p3]])

Use a Javascript callback to populate the `selected` data source as described the [callbacks documentation](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-selections).

In [43]:
source.callback = CustomJS(args=dict(selected=selected), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d1 = cb_obj.data;
        var d2 = selected.data;
        
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        selected.trigger('change');
    """)

In [44]:
show(layout)

Plot the selected data.

In [52]:
selected.data

{'x': [], 'y': []}